In [26]:
#load library
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

from __future__ import print_function

import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import norm

import tensorflow as tf

In [27]:
#set global Variables/hyperparameters
batchSize = 100
orginalDim =28*28
latentDim = 2
intermediateDim = 256
epochs= 50
epsilonSTD = 1.0

In [28]:
#Sampling function 
def sampling(args: tuple):
    zMean, zLogVar = args
    epsilon = K.random_normal(shape=(K.shape(zMean)[0],latentDim),mean=0.,stddev=epsilonSTD)
    return zMean+K.exp(zLogVar/2)*epsilon

In [29]:
#Creating encoder
x = Input(shape=(orginalDim),name="input")
h = Dense(intermediateDim,activation="relu", name="encoding")(x)
#mean of the latent space
zMean= Dense(latentDim,name="mean")(h)
#log variance of the latent space
zLogVar = Dense(latentDim, name="log-variance")(h)

z= Lambda(sampling, output_shape=(latentDim,))((zMean,zLogVar))
encoder = Model(x,[zMean,zLogVar,z], name = "encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 784)]        0           []                               
                                                                                                  
 encoding (Dense)               (None, 256)          200960      ['input[0][0]']                  
                                                                                                  
 mean (Dense)                   (None, 2)            514         ['encoding[0][0]']               
                                                                                                  
 log-variance (Dense)           (None, 2)            514         ['encoding[0][0]']               
                                                                                            

In [30]:
#Decoder
inputDecoder = Input(shape=(latentDim,),name="decoder_input")
decoderH = Dense(intermediateDim,activation="relu",name="decoder_h")(inputDecoder)
xDecoded = Dense(orginalDim,activation="sigmoid",name="flat_decoded")(decoderH)
decoder = Model(inputDecoder,xDecoded,name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 decoder_h (Dense)           (None, 256)               768       
                                                                 
 flat_decoded (Dense)        (None, 784)               201488    
                                                                 
Total params: 202,256
Trainable params: 202,256
Non-trainable params: 0
_________________________________________________________________


In [31]:
#Create Variational Autoencoder
outputCombined = decoder(encoder(x)[2])
vae = Model(x,outputCombined)
vae.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [34]:
#loss function
def vaeLoss(x: tf.Tensor, xDecodedMean: tf.Tensor,zLogVar = zLogVar,zMean=zMean,orginalDim=orginalDim):
    xentLoss = orginalDim*metrics.binary_crossentropy(x,xDecodedMean)
    klLoss = -0.5*K.sum(1+zLogVar-K.square(zMean)-K.exp(zLogVar),axis=-1)
    vaeLoss = K.mean(xentLoss+klLoss)
    return vaeLoss

vae.compile(optimizer="rmsprop",loss =vaeLoss)
vae.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________
